# Deep Learning Series

<br>
<span style="color:gray">

1. Neural Net - part.1


2. Convolution Neural Network


3. Neural Net - part.2


</span>


<b>

4. Recursive Nerural Network


</b>

# Recurrent Neural Network

## What is Recurrent Neural Network ?

RNN은 RNN이다

## Computation in Recurrent Neural Network Layer

<img src="img/RNN_01.PNG">

## BPTT (Back Propagation Through Time)

<img src="img/RNN_02.PNG">

## Gradient Vanishing & Gradient Exploding problem

## LSTM (Long-Short Term Memory)

## GRU (Gate Recurrent Unit)